In [ ]:
import pandas as pd
import numpy as np
import time 

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)  # me muestre todas las columnas
#pd.set_option('display.max_rows', None)  # me muestre todas las filas

### List of albums I am interested on

In [ ]:
topalbums = pd.read_csv('../data/clean/DB_Kaggle_top500albums_clean.csv', encoding='latin1')

topalbums.head(3)

In [ ]:
horriblealbums = pd.read_csv('../data/clean/WS_50horriblealbums_clean.csv', encoding='latin1')

horriblealbums.head(3)

In [ ]:
#need to be conscious of the fact that some albums have the same name 

topalbums['album'].value_counts().head(3)

In [ ]:
#let us try to get unique values for album_artist 

topalbums['album_artist'] = topalbums['album'] + ' - ' + topalbums['artist'] 

In [ ]:
topalbums.head(3)

In [ ]:
horriblealbums['album'].value_counts().head(2)

In [ ]:
horriblealbums['album_artist'] = horriblealbums['album'] + ' - ' + horriblealbums['artist']

In [ ]:
horriblealbums.head(2)

In [ ]:
all_albums = pd.concat([topalbums, horriblealbums], ignore_index=True)
all_albums.shape

In [ ]:
all_albums.head(550)s

In [ ]:
album_artist = all_albums['album_artist'].unique().tolist()

In [ ]:
len(album_artist)

### Spotify Connection 

In [ ]:
#%pip install pyarrow
#%pip install joblib
#%pip install tqdm
#%pip install spotipy
#%pip install python-dotenv
#%pip install requests 

In [ ]:
import requests as rq
import pandas as pd
import pyarrow
from joblib import Parallel, delayed
from tqdm import tqdm
import time
import pymongo
from passwords import *

In [ ]:
Client_ID

In [ ]:
# Obtener el TOKEN

AUTH_URL = "https://accounts.spotify.com/api/token"

creds = {
    "grant_type": "client_credentials",
    "client_id": Client_ID,
    "client_secret": Client_secret
}

response = rq.post(AUTH_URL, 
                         data=creds, 
                         headers={"Content-Type": "application/x-www-form-urlencoded"})


TOKEN = response.json()['access_token']

# Conexión con la API

url = "https://api.spotify.com/"

headers = {"Authorization": f'Bearer {TOKEN}'}

response = rq.get(url,headers=headers)
response

### Getting Album Information 

In [ ]:
albums = pd.DataFrame(all_albums['album'])
albums.head(4)

In [ ]:
# Replace spaces with the '+' character in the album names
consulta = [x.replace(' ', '+') for x in albums.album]

# Construct a list of URLs for querying album information
album_urls = [f'https://api.spotify.com/v1/search?q=album%3A{album}&type=album&limit=50&offset=0' for album in consulta]

# Create a list of dictionaries with album information and corresponding URLs
album_url_list = [{'album': album, 'url': url} for album, url in zip(albums.album, album_urls)]

In [ ]:
album_data_list = []

# Function to search and retrieve album information
def search_spotify_info(album_url):
    time.sleep(0.20)  # Pause between queries to avoid getting banned

    try:
        response = rq.get(album_url['url'], headers=headers)
        rjson = response.json()['albums']['items']
        for index, item in enumerate(rjson):
            if item['name'].lower() == album_url['album'].lower():
                album_data_list.append({
                    'Name': item['name'],
                    'Artist': item['artists'][0]['name'],
                    'Popularity': item['popularity'],
                    'Release Date': item['release_date'],
                    'Total Tracks': item['total_tracks']
                })
                return rjson[index]

        # If the album is not found, raise an exception to return None
        raise Exception
    except:
        album_data_list.append({
            'Name': None,
            'Artist': None,
            'Popularity': None,
            'Release Date': None,
            'Total Tracks': None
        })
        return {
            'album_type': None,
            'artists': None,
            'available_markets': None,
            'external_urls': None,
            'href': None,
            'id': None,
            'images': None,
            'name': None,
            'release_date': None,
            'release_date_precision': None,
            'total_tracks': None,
            'type': None,
            'uri': None
        }

# Execute the function for each album URL
for album_url in album_url_list:
    search_spotify_info(album_url)

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(album_data_list)

In [ ]:
df.head()